# <center> 3D-Deconvnet</center>

### Configuracion de grafica a usar

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="-1";

### Importacion de librerias

In [2]:
# Se importan las bibliotecas
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import cv2
import gc
from shutil import rmtree as delete
from IPython.display import clear_output
from ipywidgets import interact, IntSlider
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as     tf
from   tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from multiprocessing import Pool
from graph_functions import *
!wget -q https://raw.githubusercontent.com/JefeLitman/VideoDataGenerator/master/DatasetsLoaderUtils.py -O DatasetsLoaderUtils.py
from DatasetsLoaderUtils import flow_from_tablePaths

### Retropopagacion del modelo

In [ ]:
multiproceso = Pool()
for tipo_paciente, numero_paciente in [('CG','01'),('PG','01'),('CG','02'),('PG','02'),('CG','03'),('PG','03'),
             ('CG','04'),('PG','04'),('CG','05'),('PG','05'),('CG','06'),('PG','06'),
             ('CG','07'),('PG','07'),('CG','08'),('PG','08'),('CG','09'),('PG','09'),
             ('CG','10'),('PG','10'),('CG','11'),('PG','11')]:
    
    start_path = '/home/jefelitman/LTC_Parkinson/'
    models_path = os.path.join(start_path, 'Models/SGD_IC_Scaled_BN/',tipo_paciente+'_'+numero_paciente)
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%%% Ordenamiento de los datos %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    ori_path = os.path.join(start_path, 'data2020')
    table_paths = []
    for video in sorted(os.listdir(ori_path)):
        video_path = os.path.join(ori_path,video)
        if video.split("_")[0] == tipo_paciente and video.split("_")[1] == numero_paciente:
            table_paths.append([video_path, 'test', video.split("_")[0]])
        else:
            table_paths.append([video_path, 'train', video.split("_")[0]])
    
    def time_sampling(video):
        mitad = len(video)//2
        return video[mitad-30:mitad+30]
    def scale(video, label):
        return video/255., label
    
    dataset = flow_from_tablePaths(table_paths, time_sampling, [256, 256])
    
    test_data = tf.data.Dataset.from_generator(dataset.data_generator, 
                                                (tf.float32, tf.int64), 
                                                ([60, 256, 256, 1], []), 
                                                args=[2, 1])
    test_data = test_data.map(scale, 24).batch(1)
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%%% Obtencion de las capas de forma individual %%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    model = keras.models.load_model(os.path.join(models_path,
                                                 'final_model_patient_'+tipo_paciente+'_'+numero_paciente+'_weights.h5'))
    
    #Se nombra cada capa de la red (Ya con cada red con el mismo nombre de capas)

    # Bloque 1
    C1 = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('conv3d_1').output)
    MP1 = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('max_pooling3d_1').output)

    # Bloque 2
    C2 = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('conv3d_2').output)
    MP2 = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('max_pooling3d_2').output)

    # Bloque 3
    C3 = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('conv3d_3').output)
    MP3 = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('max_pooling3d_3').output)

    # Bloque 4
    C4 = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('conv3d_4').output)
    MP4 = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('max_pooling3d_4').output)

    # Bloque 5
    C5 = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('conv3d_5').output)
    MP5 = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('max_pooling3d_5').output)

    #Bloque 6
    Flat = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('flatten_6').output)
    FC1 = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('dense_6').output)

    #Bloque 7
    FC2 = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('dense_7').output)

    #Bloque 8
    FC3 = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('dense_8').output)
    
    folder_path = os.path.join(models_path, 'Visualizacion/')
    if os.path.isdir(folder_path):
        delete(folder_path,ignore_errors=True)
    os.mkdir(folder_path)
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%% Proceso de retropropagacion para cada video %%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    nro_video=1
    for imagen2, label in test_data:
        print('Va por el video ' + tipo_paciente + '_' + numero_paciente + '_' +str(nro_video))

        n_max = 3

        print('Inicio a convertir el video de gris a color')
        video = np.r_[[cv2.cvtColor(frame,cv2.COLOR_GRAY2BGR) for frame in imagen2.numpy()[0]]]
        video_raw = video.copy()
        heatmap = plt.cm.get_cmap('jet',n_max)
        print('Finalizo la conversion de gris a color, ahora iniciara a calcular los indices')

        for importancia in range(n_max,0,-1):
            print("Empezo calculando el indice con importancia: ",str(importancia))

            #Indice de la primera full connected
            i_max = np.argmax(FC3.predict(imagen2))

            #Indice de la segunda full connected
            x = FC2.predict(imagen2)
            w = FC3.layers[-1].weights[0].numpy()
            i_max = index_max_full_connected(i_max,w,x, importancia - 1)

            #Indice de la primera full connected
            x = FC1.predict(imagen2)
            w = FC2.layers[-1].weights[0].numpy()
            i_max = index_max_full_connected(i_max,w,x)

            #Indice de la capa flatten
            x = Flat.predict(imagen2)
            w = FC1.layers[-1].weights[0].numpy()
            i_max = index_max_full_connected(i_max,w,x)
            print('Finalizo los maximos de la fullconected, inicio con el unpooling 5')

            #Indice de la capa maxpool_5
            i_max = [np.unravel_index(i_max , (1, 1, 6, 6, 128))]
            x = C5.predict(imagen2)
            i_max = index_max_maxunpooling(i_max,x,(2,2,2),(2,2,2))
            print('Finalizo los maximos del unpooling, inicio con la convolucion 5')

            #Indice de la capa conv3d_5
            x = MP4.predict(imagen2)
            kernel = C5.layers[-1].weights[0].numpy()
            args = []
            for i in range(len(i_max)):
                args.append([i_max[i], x, kernel[:,:,:,:,i_max[i][4]], (3,3,3), (1,1,1)])
            indices = list(multiproceso.map(index_max_deconvolution_parallel,args))
            i_max = []
            for pos in indices:
                i_max += pos
            i_max = np.unique(np.r_[i_max],axis=0).tolist()
            print('Finalizo los maximos de la convolucion, inicio con el unpooling 4')

            #Indice de la capa maxpool_4
            x = C4.predict(imagen2)
            i_max = index_max_maxunpooling(i_max,x,(2,2,2),(2,2,2))
            print('Finalizo los maximos del unpooling, inicio con la convolucion 4')

            #Indice de la capa conv3d_4
            x = MP3.predict(imagen2)
            kernel = C4.layers[-1].weights[0].numpy()
            args = []
            for i in range(len(i_max)):
                args.append([i_max[i], x, kernel[:,:,:,:,i_max[i][4]], (3,3,3), (1,1,1)])
            indices = list(multiproceso.map(index_max_deconvolution_parallel,args))
            i_max = []
            for pos in indices:
                i_max += pos
            i_max = np.unique(np.r_[i_max],axis=0).tolist()
            print('Finalizo los maximos de la convolucion, inicio con el unpooling 3')

            #Indice de la capa maxpool_3
            x = C3.predict(imagen2)
            i_max = index_max_maxunpooling(i_max,x,(2,2,2),(2,2,2))
            print('Finalizo los maximos del unpooling, inicio con la convolucion 3')

            #Indice de la capa conv3d_3
            x = MP2.predict(imagen2)
            kernel = C3.layers[-1].weights[0].numpy()
            args = []
            for i in range(len(i_max)):
                args.append([i_max[i], x, kernel[:,:,:,:,i_max[i][4]], (3,3,3), (1,1,1)])
            indices = list(multiproceso.map(index_max_deconvolution_parallel,args))
            i_max = []
            for pos in indices:
                i_max += pos
            i_max = np.unique(np.r_[i_max],axis=0).tolist()
            print('Finalizo los maximos de la convolucion, inicio con el unpooling 2')

            #Indice de la capa maxpool_2
            x = C2.predict(imagen2)
            i_max = index_max_maxunpooling(i_max,x,(2,2,2),(2,2,2))
            print('Finalizo los maximos del unpooling, inicio con la convolucion 2')

            #Indice de la capa conv3d_2
            x = MP1.predict(imagen2)
            kernel = C2.layers[-1].weights[0].numpy()
            args = []
            for i in range(len(i_max)):
                args.append([i_max[i], x, kernel[:,:,:,:,i_max[i][4]], (3,3,3), (1,1,1)])
            indices = list(multiproceso.map(index_max_deconvolution_parallel,args))
            i_max = []
            for pos in indices:
                i_max += pos
            i_max = np.unique(np.r_[i_max],axis=0).tolist()
            print('Finalizo los maximos de la convolucion, inicio con el unpooling 1')

            #Indice de la capa maxpool_1
            x = C1.predict(imagen2)
            i_max = index_max_maxunpooling(i_max,x,(1,2,2),(1,2,2))
            print('Finalizo los maximos del unpooling, inicio con la convolucion 1')

            #Indice de la capa conv3d_1
            x = imagen2.numpy()
            kernel = C1.layers[-1].weights[0].numpy()
            args = []
            for i in range(len(i_max)):
                args.append([i_max[i], x, kernel[:,:,:,:,i_max[i][4]], (3,3,3), (1,1,1)])
            indices = list(multiproceso.map(index_max_deconvolution_parallel,args))
            i_max = []
            for pos in indices:
                i_max += pos
            i_max = np.unique(np.r_[i_max],axis=0)
            print('Finalizo los maximos de la convolucion, total indices maximos: '+str(len(i_max)))

            del indices

            #Pinto los puntos en el video
            print("Ahora va iniciar a pintar los puntos de importancia "+str(importancia)+" en el video")
            args = []
            for i in range(video.shape[0]):
                args.append([video[i], i, i_max, heatmap, importancia, n_max])
            video = np.r_[list(multiproceso.map(graph_indexes_on_video_parallel,args))]
            print("Termino de pintar todos los puntos en el video, ahora se va empezar a calcular los siguientes puntos")
            del args

        print("Termino de pintar todos los puntos en el video. Ahora va empezar a guardarlo")
        video_path = os.path.join(folder_path,str(nro_video))
        os.mkdir(video_path)

        img = video_raw[0]
        for frame in range(len(video)):
            # Save figure.
            if (frame+1) % 15 == 0:
                img = cv2.addWeighted(video_raw[frame],0.5,img,0.5,0)
            video[frame] = cv2.addWeighted(img,0.4,video[frame],0.6,0)
            video[frame] = cv2.cvtColor(video[frame],cv2.COLOR_BGR2RGB)
            fig, ax = plt.subplots(1)
            im = ax.imshow(video[frame], cmap='jet')
            ax.set_axis_off()
            fig.colorbar(im)
            if frame+1<10:
                #cv2.imwrite("salida/frame_00"+str(frame+1)+'.png', video[frame])
                plt.savefig(video_path+"/frame_00"+str(frame+1)+'.png', bbox_inches = 'tight', pad_inches = 0)
            else:
                #cv2.imwrite("salida/frame_0"+str(frame+1)+'.png', video[frame])
                plt.savefig(video_path+"/frame_0"+str(frame+1)+'.png', bbox_inches = 'tight', pad_inches = 0)
            plt.close()
        print("Termino de guardar todo el video "+ tipo_paciente + '_' + numero_paciente + '_' +str(nro_video))
        nro_video+=1
        clear_output()
    print("Termino todos los videos del paciente "+ tipo_paciente + '_' + numero_paciente)
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # Libero la memoria
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    del model
    del dataset
    del table_paths
    del fig
    del ax
    del C1
    del MP1
    del C2
    del MP2
    del C3 
    del MP3
    del C4
    del MP4
    del C5
    del MP5
    del Flat
    del FC1
    del FC2
    del FC3
    keras.backend.clear_session()
    gc.collect()

Va por el video PG_05_2
Inicio a convertir el video de gris a color
Finalizo la conversion de gris a color, ahora iniciara a calcular los indices
Empezo calculando el indice con importancia:  3
Finalizo los maximos de la fullconected, inicio con el unpooling 5
Finalizo los maximos del unpooling, inicio con la convolucion 5
Finalizo los maximos de la convolucion, inicio con el unpooling 4
Finalizo los maximos del unpooling, inicio con la convolucion 4
Finalizo los maximos de la convolucion, inicio con el unpooling 3
Finalizo los maximos del unpooling, inicio con la convolucion 3
Finalizo los maximos de la convolucion, inicio con el unpooling 2
Finalizo los maximos del unpooling, inicio con la convolucion 2
Finalizo los maximos de la convolucion, inicio con el unpooling 1
Finalizo los maximos del unpooling, inicio con la convolucion 1
Finalizo los maximos de la convolucion, total indices maximos: 154005
Ahora va iniciar a pintar los puntos de importancia 3 en el video
Termino de pintar t

### Momento de realizar la matrix de confusion sobre los datos de test

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    import itertools
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if cmap is None:
        cmap = plt.get_cmap('Blues')
    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
y_true = []
y_pred = []
for tipo_paciente, numero_paciente in [('CG','01'),('PG','01'),('CG','02'),('PG','02'),('CG','03'),('PG','03'),
                                    ('CG','04'),('PG','04'),('CG','05'),('PG','05'),('CG','06'),('PG','06'),
                                    ('CG','07'),('PG','07'),('CG','08'),('PG','08'),('CG','09'),('PG','09'),
                                    ('CG','10'),('PG','10'),('CG','11'),('PG','11')]:
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%%% Ordenamiento de los datos %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    ds_path = 'data'
    
    delete('data',ignore_errors=True)
    os.mkdir('data')

    ori_path = 'data2020'

    for modo in ["train", "test"]:
        modo_path = os.path.join(ds_path,modo)
        os.mkdir(modo_path)
        for clase in ['CG','PG']:
            clase_path = os.path.join(modo_path,clase)
            os.mkdir(clase_path)

    for video in sorted(os.listdir(ori_path)):
        video_path = os.path.join(ori_path,video)
        if video.split("_")[0] == tipo_paciente and video.split("_")[1] == numero_paciente:
            dst_path = os.path.join(ds_path,'test',video.split("_")[0],video)
        else:
            dst_path = os.path.join(ds_path,'train',video.split("_")[0],video)
        os.mkdir(dst_path)
        for frame in sorted(os.listdir(video_path)):
            frame_ori = os.path.join(video_path,frame)
            copy(frame_ori,dst_path)
    
    def half_video_temporal(frames):
        mitad = len(frames)//2
        return [frames[mitad-30:mitad+30]]
    def video_transf(video):
        escalador = MinMaxScaler()
        new_video = video.reshape((video.shape[0]*video.shape[1]*video.shape[2]*video.shape[3],1))
        new_video = escalador.fit_transform(new_video)
        return new_video.reshape((video.shape[0],video.shape[1],video.shape[2],video.shape[3]))
    dataset = VideoDataGenerator(directory_path = ds_path, 
                            batch_size = 4, 
                            original_frame_size = [520,520], 
                            frame_size= [256,256], 
                            video_frames = 60, 
                            temporal_crop = ("custom", half_video_temporal),
                            video_transformation = video_transf,
                            frame_crop = (None, None), 
                            shuffle = False, 
                            conserve_original = False)
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%% Obtencion del modelo entrenado %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    model = keras.models.load_model('/home/jefelitman/LTC_Parkinson/Models/final_model_patient_'+tipo_paciente+'_'+numero_paciente+'_weights.h5')
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%%5%% Proceso de almacenar los y_true e y_pred %%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    for _ in range(2):
        batch, labels = dataset.get_next_test_batch(1)
        prediction = model.predict(batch)
        y_pred = np.concatenate((y_pred,np.argmax(prediction,axis=1)))
        y_true = np.concatenate((y_true,labels))
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Pinto la matriz de confusion %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
plot_confusion_matrix(cm = confusion_matrix(y_true,y_pred),
                      target_names = dataset.to_class,
                      title='Confusion matrix',
                      cmap=None,
                      normalize=False)